### Declare your keys from your TWITTER DEVELOPER PORTAL

In [ ]:
# you need to install these 3rd party libraries using command prompt
#!pip install tweepy
#!pip install botometer

In [ ]:
import tweepy
import botometer
import pandas as pd

# get your RAPIDAPI keys here https://rapidapi.com/
# TWITTER DEVELOPER KEYS here https://developer.twitter.com/
rapidapi_key = "xxxxxxxxxxxxxxxxxxxxxx"
consumer_key = r"yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy"
consumer_secret = "zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz"
access_token = r"aaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
access_token_secret = r"bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

### Let us declare target TwitterId , which is BONGBONG MARCOS' and LENI ROBREDO's

In [ ]:
leni_user = api.get_user("lenirobredo") 
print("TwitterID: " + leni_user.id_str + "| ScreenName: " + leni_user.screen_name)

In [ ]:
bbm_user = api.get_user("bongbongmarcos") 
print("TwitterID: " + bbm_user.id_str + "| ScreenName: " + bbm_user.screen_name)

In [ ]:
user_ids = []
user_ids.append(leni_user.id_str)
user_ids.append(bbm_user.id_str)

### Here's another way to get Twitter Users by passing a list of twitter IDs

In [ ]:
user_collection = [user for user in api.lookup_users(user_ids)]

### We Initialize The Botometer object

In [ ]:
twitter_app_auth = {
        "consumer_key": consumer_key
        ,"consumer_secret": consumer_secret
        ,"access_token":access_token
        ,"access_token_secret": access_token_secret
  }
bom = botometer.Botometer(wait_on_ratelimit=True,
                          rapidapi_key=rapidapi_key,
                          **twitter_app_auth)

### This is a simple JSON flattener

In [ ]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

## Let's Run bongbongmarcos' and LeniRobredo's twitter accounts against Botometer

In [ ]:
import json

bom = botometer.Botometer(wait_on_ratelimit=True,
                          rapidapi_key=rapidapi_key,
                          **twitter_app_auth)

bom_results = []
for screen_name, result in bom.check_accounts_in(user_ids):
    bom_results.append(flatten_json(result))

### We Flatten the JSONified results and insert them as individual rows in a pandas dataframe

In [ ]:
df_bom_results = []

for result in bom_results:
    #print(result)
    df =pd.DataFrame(result.items(), columns=['Index', 'Value']).T

    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data less the header row
    df.columns = new_header #set the header row as the df header
    df.reset_index(drop=True, inplace=True)
    df_bom_results.append(df)

In [ ]:
len(df_bom_results)

### Here's how to interpret the results https://rapidapi.com/OSoMe/api/botometer-pro/details

In [ ]:
df_combined = pd.concat(df_bom_results, axis=0)
df_combined.reset_index(drop=True, inplace=True)
df_combined.head(5)